In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Installing Haystack and malaya

In [ ]:
!pip install malaya

In [ ]:
!pip install sacremoses

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 14.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895241 sha256=4f61d98fa2b2cea6a35a29cd150faa9320da917c840e9289675bcfaabc234c77
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully built sacremoses


In [ ]:
%%bash

pip install --upgrade pip
pip install farm-haystack[colab]

In [ ]:
!pip install transformers
!pip install torch
!pip install -U sentence-transformers

In [ ]:
!pip install langdetect

Read Document

In [ ]:
import json

documents = []
path = '/content/drive/Shareddrives/NLP/Project/documents_v2.json'
path = '/content/drive/MyDrive/Trimester 2 2022 2023/TNL3221 Natural Language Processing/Project/documents_v2.json'
# Read all documents from the local JSON file
with open(path, 'r', encoding='utf-8') as f:
    docs = json.load(f)


In [ ]:
documents = []
for i in range(len(docs)) :
  documents.append(docs[i]['content'])

Translate Ques to English

In [ ]:
# translate ques to eng
import malaya
from langdetect import detect
from transformers import MarianMTModel, MarianTokenizer

model_de = 'Helsinki-NLP/opus-mt-de-en'
model_fr = 'Helsinki-NLP/opus-mt-fr-en'
model_it = 'Helsinki-NLP/opus-mt-it-en'

tokenizer_de = MarianTokenizer.from_pretrained(model_de)
model_de = MarianMTModel.from_pretrained(model_de)
tokenizer_fr = MarianTokenizer.from_pretrained(model_fr)
model_fr = MarianMTModel.from_pretrained(model_fr)
tokenizer_it = MarianTokenizer.from_pretrained(model_it)
model_it = MarianMTModel.from_pretrained(model_it)

#transformer_ms = malaya.translation.ms_en.transformer()
transformer_small_ms = malaya.translation.ms_en.transformer(model = 'small')


In [ ]:
# translate
def translate_ms_en(text):
    return transformer_small_ms.greedy_decoder([text])

def translate_ques_to_eng(text):
    lang = detect(text)

    if lang == 'de':
        # Use German to English model
        inputs = tokenizer_de(text, return_tensors="pt", padding=True)
        translated = model_de.generate(**inputs)
        translated_text = [tokenizer_de.decode(t, skip_special_tokens=True) for t in translated]
        print()
    elif lang == 'fr':
        # Use French to English model
        inputs = tokenizer_fr(text, return_tensors="pt", padding=True)
        translated = model_fr.generate(**inputs)
        translated_text = [tokenizer_fr.decode(t, skip_special_tokens=True) for t in translated]
    elif lang == 'it':
        # Use Italian to English model
        inputs = tokenizer_it(text, return_tensors="pt", padding=True)
        translated = model_it.generate(**inputs)
        translated_text = [tokenizer_it.decode(t, skip_special_tokens=True) for t in translated]
    elif lang == 'ms' or lang == 'id' or lang == 'id':
        # Use Malay to English model
        translated_text = translate_ms_en(text)
        #translated_text = tokenizer_ms.tokenize(translated_text)
    else:
        # If the language is neither of the above languages, return the original text
        return text
    return translated_text[0]

In [ ]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Tokenize sentences
nltk.download('punkt') # first-time use only
nltk.download('stopwords') # first-time use only

# Preprocessing
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

stop_words = set(stopwords.words('english'))

def preprocess_ques(doc):
    # check if the document is in english
    if detect(doc) != 'en':
      doc = translate_ques_to_eng(doc)
    word_tokens = word_tokenize(doc)
    # lowercase
    word_tokens = [word.lower() for word in word_tokens]
    # remove stopwords
    filtered_text = [word for word in word_tokens if word.casefold() not in stop_words]
    # Remove punctuation
    punctuations = ['(', ')', ';',':','"']
    filtered_text = [word for word in filtered_text if filtered_text not in punctuations]
    return ' '.join(filtered_text)

# Filter out non-string documents
documents = [doc for doc in documents if isinstance(doc, str)]

# Now preprocess the documents
# documents = [preprocess_text(doc) for doc in documents]


In [ ]:
# third model # best model
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Initialize the BERT model (this will download the model the first time)
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

# Generate embeddings for each document
embeddings = model.encode(documents)

def ask_question(query):
    query = preprocess_ques(query)

    # Generate the BERT embedding for the query
    query_embedding = model.encode([query])

    # Compute the cosine similarity between the query embedding and all document embeddings
    results = cosine_similarity(embeddings, query_embedding).flatten()

    # Get the index of the most similar document
    most_similar_doc_index = results.argsort()[-1]

    return documents[most_similar_doc_index]

# Testing
ques = "quand est né barack obama?"
answer = ask_question(ques)
print(answer)

Translate Answer to Malay

In [ ]:
# translate ques to eng
import malaya
from langdetect import detect
from transformers import MarianMTModel, MarianTokenizer

model_en_de = 'Helsinki-NLP/opus-mt-en-de'
model_en_fr = 'Helsinki-NLP/opus-mt-en-fr'
model_en_it = 'Helsinki-NLP/opus-mt-en-it'

tokenizer_en_de = MarianTokenizer.from_pretrained(model_en_de)
model_en_de = MarianMTModel.from_pretrained(model_en_de)
tokenizer_en_fr = MarianTokenizer.from_pretrained(model_en_fr)
model_en_fr = MarianMTModel.from_pretrained(model_en_fr)
tokenizer_en_it = MarianTokenizer.from_pretrained(model_en_it)
model_en_it = MarianMTModel.from_pretrained(model_en_it)

#transformer_ms = malaya.translation.ms_en.transformer()
transformer_small = malaya.translation.en_ms.transformer(model = 'small')

In [ ]:
# translate
def translate_en_ms(text):
    return transformer_small.greedy_decoder([text])

def translate_ans_to_eng(ques,ans):
    lang = detect(ques)

    if lang == 'de':
        # Use English to German model
        inputs = tokenizer_en_de(ans, return_tensors="pt", padding=True)
        translated = model_en_de.generate(**inputs)
        translated_text = [tokenizer_en_de.decode(t, skip_special_tokens=True) for t in translated]
    elif lang == 'fr':
        # Use English to French model
        inputs = tokenizer_en_fr(ans, return_tensors="pt", padding=True)
        translated = model_en_fr.generate(**inputs)
        translated_text = [tokenizer_en_fr.decode(t, skip_special_tokens=True) for t in translated]
    elif lang == 'it':
        # Use English to Italian model
        inputs = tokenizer_en_it(ans, return_tensors="pt", padding=True)
        translated = model_en_it.generate(**inputs)
        translated_text = [tokenizer_en_it.decode(t, skip_special_tokens=True) for t in translated]
    elif lang == 'ms' or lang == 'id' or lang == 'id':
        # Use English to Malay model
        translated_text = translate_en_ms(ans)
        #translated_text = tokenizer_ms.tokenize(translated_text)
    else:
        # If the language is neither of the above languages, return the original text
        return ans
    return translated_text[0]

In [ ]:
translated_answer = translate_ans_to_eng(ques, answer)
print("EN Answer:\n", answer)
print("Translated Answer:\n", translated_answer)

Deploy the question answering system on a Fast API

In [ ]:
!pip install fastapi uvicorn pyngrok

In [ ]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pyngrok import ngrok
import uvicorn
import nest_asyncio

app = FastAPI()

origins = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Endpoint to get an answer to a question
@app.get('/get_answer/{question}')
async def get_answer(question: str):
    # Get the answer to the question
    answer = ask_question(question)

    # Translate the answer to the language of the question
    final_answer = translate_ans_to_eng(question,answer)
    return final_answer

# Endpoint to check the connection status
@app.get('/check')
async def check():
    return 'connected'

# Connect to ngrok for tunneling
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)